In [2]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pip setuptools wheel

  Using cached setuptools-78.1.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached setuptools-78.1.0-py3-none-any.whl (1.3 MB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.6.0
    Uninstalling setuptools-75.6.0:
      Successfully uninstalled setuptools-75.6.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 9.5 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install gpy

  Using cached paramz-0.9.6-py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.0 MB/s eta 0:00:00
Using cached paramz-0.9.6-py3-none-any.whl (103 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 11.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
'''
This python script performs 10-fold cross validation for a dataset for GP machine learning method and outputs the resulting average metrics.
The metrics are calculated for each of the 10 folds then the mean taken as the final metrics.
In addition to the predictions, an upper and lower prediction (or error) is also calculated based on the prediction that encompasses 1 SD.
INPUTS:
Dataset.csv - a .csv file with the data in. The columns must be named in the same way as below
OUTPUTS:
GPCV_metrics - a .csv file of the metrics for GP method for its performance using 10-fold CV. "Max % within" refers to whether the prediction, with the upper and lower levels, fall within the range.
GPCV_predictions - a .csv file containing the actual and predicted LogS values for each compound
'''

#section 1: import modules
import sklearn
from sklearn.model_selection import KFold
import statistics
import sys,os,re
import pandas as pd
import numpy as np
from sklearn import preprocessing
import GPy
from scipy.stats import pearsonr
import math

##section 2: define inputs and outputs
dir = os.getcwd() 
Dataset=pd.read_csv(os.path.join(dir,"/Users/stella/Downloads/tierney/project/all_data_final.csv"))
output_metrics=os.path.join(dir,"/Users/stella/Downloads/tierney/project/GPCV_metrics.csv")
output_predictions=os.path.join(dir,"/Users/stella/Downloads/tierney/project/GPCV_predictions.csv")

##section 3: define methods
#Define statistical measures and R2 conversion
#define RMSE
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
#define method to find predictions within certain range
def within_range(list1, list2, range2):
    x=0
    for i in range(len(list2)):
        if (list1[i]-range2)<= list2[i] <= (list1[i]+range2): 
            x+=1
    return((float(x)/(len(list2)))*100)
def within_range_errors(list1, list2, list3, list4, range2):
    x=0
    for i in range(len(list2)):
        if (list1[i]-range2)<= list2[i] <= (list1[i]+range2): 
            x+=1
        elif (list1[i]-range2)<= list3[i] <= (list1[i]+range2): 
            x+=1
        elif (list1[i]-range2)<= list4[i] <= (list1[i]+range2): 
            x+=1
    return((float(x)/(len(list2)))*100)
#define getting R2 method
def get_R2(R2):
    R2_2=[]
    for i in range(len(R2)):
        x=re.findall('\d\.\d+',str(R2[i]))
        j=float(x[0])
        j=j**2
        R2_2.append(j)
    return(R2_2)
#define method to get CV results
def CV_metrics(Data,folds):
    #initiate lists to add metrics to
    RMSE=[]
    R2=[]
    N1=[]
    N05=[]
    N1_e=[]
    N05_e=[]
    GPR_RMSE=[]
    GPR_R2=[]
    GPR_N1=[]
    GPR_N05=[]
    GPR_N1_e=[]
    GPR_N05_e=[]
    
    # Initialize list to store predictions and actual values
    all_predictions = []
    
    #import Data
    X = Data
    X = X.sample(frac=1).reset_index(drop=True)
    # Store original indices to identify compounds later
    X['original_index'] = X.index
    #define k-fold cross validation
    col_names=X.dtypes.index
    X = np.array(X)
    kf = KFold(n_splits=folds)
    
    for fold, (train1, test1) in enumerate(kf.split(X)):
        train=X[train1]
        test=X[test1]
        train=pd.DataFrame(data=train, columns=col_names)
        test=pd.DataFrame(data=test, columns=col_names)
        
        # Store test indices to identify compounds
        test_indices = test['original_index'].values
        
        X_train = train[['MW','Volume','G_sol','DeltaG_sol','sol_dip',
                     'LsoluHsolv','LsolvHsolu','SASA','O_charges',
                     'C_charges','Most_neg','Most_pos','Het_charges']]
        y_train = train['LogS']
        X_test = test[['MW','Volume','G_sol','DeltaG_sol','sol_dip',
                     'LsoluHsolv','LsolvHsolu','SASA','O_charges',
                     'C_charges','Most_neg','Most_pos','Het_charges']]
        y_test = test['LogS']
        
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        y_train=[[i] for i in y_train]
        y_train=np.array(y_train)
        
        #run models
        kernel = GPy.kern.RBF(input_dim=13, variance=1., lengthscale=1.)
        GPR=GPy.models.GPRegression(X_train,y_train,kernel)
        GPR.optimize()
        gpr2preds = GPR.predict(X_test)[0]
        errors=GPR.predict_quantiles(X_test,quantiles=(16,84)) # 1 SD confidence interval
        
        gpr2preds=[i[0] for i in gpr2preds]
        errors[0]=[i[0] for i in errors[0]]
        errors[1]=[i[0] for i in errors[1]]
        
        # Store predictions for this fold
        for i in range(len(test_indices)):
            all_predictions.append({
                'original_index': test_indices[i],
                'fold': fold+1,
                'actual': y_test.iloc[i],
                'predicted': gpr2preds[i],
                'lower_bound': errors[0][i],
                'upper_bound': errors[1][i]
            })
        
        #evaluate model
        GPR_R2.append(pearsonr(gpr2preds, y_test))
        GPR_RMSE.append(rmse(gpr2preds, y_test))
        GPR_N1.append(within_range(y_test,gpr2preds,1))
        GPR_N05.append(within_range(y_test,gpr2preds,0.7))
        GPR_N1_e.append(within_range_errors(y_test,gpr2preds,errors[0],errors[1],1))
        GPR_N05_e.append(within_range_errors(y_test,gpr2preds,errors[0],errors[1],0.7))
    
    # Convert predictions to DataFrame
    predictions_df = pd.DataFrame(all_predictions)
    
    #get R2 from Pearson output
    GPR_R2=get_R2(GPR_R2)
    #get mean metrics and put together in lists
    R2.append(statistics.mean(GPR_R2))
    RMSE.append(statistics.mean(GPR_RMSE))
    N1.append(statistics.mean(GPR_N1))
    N05.append(statistics.mean(GPR_N05))
    N1_e.append(statistics.mean(GPR_N1))
    N05_e.append(statistics.mean(GPR_N05))
    
    #create dataframe of metrics
    Models=["GPR"]
    Metrics=list(zip(Models,R2,RMSE,N1,N05,N1_e,N05_e))
    Metrics_df=pd.DataFrame(data=Metrics, columns=['Model','R2','RMSE','% within 1','% within 0.7','Max % within 1','Max % within 0.7'])
    
    return Metrics_df, predictions_df

##method to put it all together
def get_CV_metrics(Dataset, output_metrics, output_predictions):
    ##get metrics and predictions
    CV_metrics_df, predictions_df = CV_metrics(Dataset, 10)  ##10-folds
    
    ##save to files
    CV_metrics_df.to_csv(output_metrics, index=False)
    predictions_df.to_csv(output_predictions, index=False)

##section 4: run CV method and get metrics and predictions
get_CV_metrics(Dataset, output_metrics, output_predictions)

 <>:56: SyntaxWarning:invalid escape sequence '\d'
 <>:56: SyntaxWarning:invalid escape sequence '\d'
 /var/folders/m2/ptj22ckn1lq9dwc6k6bwvz580000gn/T/ipykernel_33347/2968771801.py:56: SyntaxWarning:invalid escape sequence '\d'
